# Basic PUMS Analysis with WhiteNoise

This notebook will be a brief tutorial on doing data analysis within the WhiteNoise system.

We will start out by setting up our environment -- loading the necessary libraries and establishing the very basic things we need to know before loading our data (the file path and variable names).

In [1]:
# load libraries
import os
import sys
import numpy as np
import opendp.whitenoise.core as wn

# establish data information
data_path = os.path.join('.', 'data', 'PUMS_california_demographics_1000', 'data.csv')
var_names = ["age", "sex", "educ", "race", "income", "married", "pid"]

### Properties

The core WhiteNoise library is made up of two key pieces; the runtime and the validator. The runtime is made up of low-level algorithms and operations. The validator contains logic for combining runtime elements into more complex operations, as well as methods for determining whether or not a computation is differentially private. If an analysis plan is deemed to produce data that are not differentially private, the validator will not allow this analysis to run. Importantly, this is done independent of the underlying data.

Whether or not a set of computations produces differentially private data relies on a set of properties of the data. These properties can be statically determined (without touching the actual data) and can be updated at each step of the analysis. One pair of common properties is `lower/upper`. For a differentially private mean, for example, the validator requires the input data to have defined lower and upper bounds. An analyst can ensure that `lower/upper` are set with the `clamp` component, which clamps data to a given range.  

Let's say that we have access to the PUMS codebook, and thus know some basic information about the possible values for the variables in the data. This is a convenient way to have reasonable baselines for properties like `lower/upper`.

Another common property is `n`, an estimate of the sample size of the data in question. In general, this could be based on true knowledge of the data, an educated guess, or we could produce it via a differentially private process. We know, by construction of the data set, that this is a 1,000 person sample.

Yet another property is `nullity`, whether or not the validator can guarantee that results are not null. We will see what it looks like for both the `lower/upper` and `nullity` properties to change within an analysis. 

Let's start with `lower/upper`.

In [2]:
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)
    
    # establish data 
    age_dt = wn.to_float(data['age'])

    # clamp data to set lower/upper properties
    clamped_age_dt = wn.clamp(age_dt, lower = 0., upper = 100.)

    # expand lower/upper by a factor of 2
    clamped_age_dt_2 = wn.multiply(clamped_age_dt, 2.)


analysis.release()
print('original properties:\n{0}\n\n'.format(age_dt.properties))
print('properties after clamping:\n{0}\n\n'.format(clamped_age_dt.properties))
print('properties after multiplication:\n{0}\n\n'.format(clamped_age_dt_2.properties))

original properties:
array {
  num_records {
  }
  num_columns {
    option: 1
  }
  nullity: true
  c_stability {
    data: 1.0
  }
  data_type: F64
  dataset_id {
    option: 0
  }
  is_not_empty: true
  dimensionality: 1
}



properties after clamping:
array {
  num_records {
  }
  num_columns {
    option: 1
  }
  nullity: true
  c_stability {
    data: 1.0
  }
  data_type: F64
  dataset_id {
    option: 0
  }
  is_not_empty: true
  dimensionality: 1
  continuous {
    minimum {
      f64 {
        data {
          option: 0.0
        }
      }
    }
    maximum {
      f64 {
        data {
          option: 100.0
        }
      }
    }
  }
}



properties after multiplication:
array {
  num_records {
    option: 1
  }
  num_columns {
    option: 1
  }
  nullity: true
  c_stability {
    data: 1.0
  }
  data_type: F64
  dataset_id {
    option: 0
  }
  is_not_empty: true
  dimensionality: 1
  continuous {
    minimum {
      f64 {
        data {
          option: 0.0
        }
   

We can now move onto `nullity`.

In [3]:
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)
    
    # establish data 
    age_dt = wn.to_float(data['age'])

    # ensure data are non-null 
    non_null_age_dt = wn.impute(age_dt, distribution = 'Uniform', lower = 0., upper = 100.)

    # create null data
    potentially_null_age_dt = wn.multiply(non_null_age_dt, float('nan'))

analysis.release()
print('original properties:\n{0}\n\n'.format(age_dt.properties))
print('properties after imputation:\n{0}\n\n'.format(non_null_age_dt.properties))
print('properties after multipliction by nan:\n{0}\n\n'.format(potentially_null_age_dt.properties))


original properties:
array {
  num_records {
  }
  num_columns {
    option: 1
  }
  nullity: true
  c_stability {
    data: 1.0
  }
  data_type: F64
  dataset_id {
    option: 0
  }
  is_not_empty: true
  dimensionality: 1
}



properties after imputation:
array {
  num_records {
  }
  num_columns {
    option: 1
  }
  c_stability {
    data: 1.0
  }
  data_type: F64
  dataset_id {
    option: 0
  }
  is_not_empty: true
  dimensionality: 1
  continuous {
    minimum {
      f64 {
        data {
        }
      }
    }
    maximum {
      f64 {
        data {
        }
      }
    }
  }
}



properties after multipliction by nan:
array {
  num_records {
    option: 1
  }
  num_columns {
    option: 1
  }
  nullity: true
  c_stability {
    data: 1.0
  }
  data_type: F64
  dataset_id {
    option: 0
  }
  is_not_empty: true
  dimensionality: 1
  continuous {
    minimum {
      f64 {
        data {
        }
      }
    }
    maximum {
      f64 {
        data {
        }
      }
    }


Note that the `nullity` property disappears after imputation (`nullity` disappearing is equivalent to `nullity: false`) and reappears after multiplication by `nan`. 

### Analysis

Now we can proceed to performing a basic analysis. Let's start by considering a differentially private mean of `age`. We will start with a few failed attempts in order to build an intuition for the requisite steps.

In [4]:
# set sample size
n = 1_000

# set ranges/feasible values
age_range = (0., 100.)
sex_vals = [0, 1]
educ_vals = [i for i in range(1, 17)]
race_vals = [i for i in range(1, 7)]
income_range = (0., 500_000.)
married_vals = [0, 1]

In [5]:
# attempt 1 - fails because of nullity
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)
    
    ''' get mean age '''
    # establish data 
    age_dt = wn.to_float(data['age'])
    
    # calculate differentially private mean of age
    age_mean = wn.dp_mean(data = age_dt, privacy_usage={'epsilon': .65})

analysis.release()

Error: at node_id 5
Caused by: node specification LaplaceMechanism(LaplaceMechanism { privacy_usage: [PrivacyUsage { distance: Some(Approximate(DistanceApproximate { epsilon: 0.65, delta: 0.0 })) }] }):
Caused by: node specification Mean(Mean):
Caused by: data may contain nullity when non-nullity is required



/Users/michael/whitenoise/python/opendp/whitenoise/core/base.py:674: UserWarning: Some nodes were not allowed to execute.
  warnings.warn("Some nodes were not allowed to execute.")


Notice that `dp_mean` requires the data to have the property `nullity = False`.
We can get around this by using `impute`. We will impute from a `Gaussian(mean = 45, sd = 10)` distribution, truncated such that no values fall outside of our age range we already established.

In [6]:
# attempt 2 - fails because of undefined lower/upper
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)
    
    ''' get mean age '''
    # establish data 
    age_dt = wn.to_float(data['age'])
    
    # clamp data to range and impute missing values
    age_dt = wn.impute(data = age_dt, distribution = 'Gaussian',
                                      lower = age_range[0], upper = age_range[1],
                                      shift = 45., scale = 10.)
    
    # calculate differentially private mean of age
    age_mean = wn.dp_mean(data = age_dt, privacy_usage={'epsilon': .65})
     
analysis.release()

Error: at node_id 11
Caused by: node specification LaplaceMechanism(LaplaceMechanism { privacy_usage: [PrivacyUsage { distance: Some(Approximate(DistanceApproximate { epsilon: 0.65, delta: 0.0 })) }] }):
Caused by: node specification Mean(Mean):
Caused by: not all lower bounds are known



Now we see that `dp_mean` needs to know the `lower` value (in fact, it also needs to know `upper`). We provide that with `clamp`. We paramaterize `clamp` with the lower and upper values of age we established at the beginning.

In [7]:
# attempt 3 - fails because of undefined n
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)
    
    ''' get mean age '''
    # establish data 
    age_dt = wn.to_float(data['age'])
    
    # clamp data to range and impute missing values
    age_dt = wn.clamp(data = age_dt, lower = age_range[0], upper = age_range[1])
    age_dt = wn.impute(data = age_dt, distribution = 'Gaussian',
                                      lower = age_range[0], upper = age_range[1],
                                      shift = 45., scale = 10.)
    
    # calculate differentially private mean of age
    age_mean = wn.dp_mean(data = age_dt, privacy_usage={'epsilon': .65})

    
analysis.release()

Error: at node_id 14
Caused by: node specification LaplaceMechanism(LaplaceMechanism { privacy_usage: [PrivacyUsage { distance: Some(Approximate(DistanceApproximate { epsilon: 0.65, delta: 0.0 })) }] }):
Caused by: node specification Mean(Mean):
Caused by: number of records is not defined



WhiteNoise requires `n` to be specified before a mean release can be considered valid.
We know the true `n` in this case, but this will not always be true. We call `resize` to ensure that the data are consistent with the `n` we provide.

In [8]:
# attempt 4 - succeeds!
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)
    
    ''' get mean age '''
    # establish data 
    age_dt = wn.to_float(data['age'])
    
    # clamp data to range and impute missing values
    age_dt = wn.clamp(data = age_dt, lower = age_range[0], upper = age_range[1])
    age_dt = wn.impute(data = age_dt, distribution = 'Gaussian',
                                      lower = age_range[0], upper = age_range[1],
                                      shift = 45., scale = 10.)
    
    # ensure data are consistent with proposed n
    age_dt = wn.resize(data = age_dt, n = n, distribution = 'Gaussian',
                       lower = age_range[0], upper = age_range[1],
                       shift = 45., scale = 10.)
    
    # calculate differentially private mean of age
    age_mean = wn.dp_mean(data = age_dt, privacy_usage={'epsilon': .65})
        
    ''' get variance of age '''
    # calculate differentially private variance of income
    age_var = wn.dp_variance(data = age_dt, privacy_usage={'epsilon': .35})
    
analysis.release()

# print differentially private estimates of mean and variance of age
print(age_mean.value)
print(age_var.value)

44.945627965690974
339.91407708915904


Notice that we asked for an extra `dp_variance` at the end without having to use `clamp`, `impute`, or `resize`. Because these functions are updating the properties of `age_dt` as they are called, `dp_variance` has everything it needs from `age_dt` when we call it.

Now that we have a sense for building up a statistic step-by-step, we can run through a much quicker version. We simply provide `data_lower, data_upper, data_n` and the `clamp, impute, resize` steps are all performed implicitly. You'll notice that we don't even provide a `distribution` argument, even though it is needed for `impute`. For some arguments, we have (what we believe to be) reasonable defaults that are used if not provided explicitly. 

In [9]:
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)
    
    # cast to float
    age_dt = wn.to_float(data['age'])
    
    # get mean of age
    age_mean = wn.dp_mean(data = age_dt,
                          privacy_usage = {'epsilon': .65},
                          data_lower = 0.,
                          data_upper = 100.,
                          data_n = 1000
                         )
    # get variance of age
    age_var = wn.dp_variance(data = age_dt,
                             privacy_usage = {'epsilon': .35},
                             data_lower = 0.,
                             data_upper = 100.,
                             data_n = 1000
                            )
analysis.release()

print("DP mean of age: {0}".format(age_mean.value))
print("DP variance of age: {0}".format(age_var.value))
print("Privacy usage: {0}".format(analysis.privacy_usage))

DP mean of age: 44.68970580961599
DP variance of age: 306.5102273650341
Privacy usage: approximate {
  epsilon: 1.0
}



We see that the two DP releases within our analysis compose in a simple way, the individual epsilons we set add together for a total privacy usage of 1.   

One thing we have glossed over up until this point is the distinction between setting up and executing an analysis plan. An analysis plan is specified within the encapsulation of `wn.Analysis()` but until `analysis.release()` is run, the plan will not be validated and the data will not be touched.

In [10]:
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)

    # get mean of age
    age_mean = wn.dp_mean(data = wn.to_float(data['age']),
                          privacy_usage = {'epsilon': .65},
                          data_lower = 0.,
                          data_upper = 100.,
                          data_n = 1000
                         )
print("Pre-Release\n")
print("DP mean of age: {0}".format(age_mean.value))
print("Privacy usage: {0}\n\n".format(analysis.privacy_usage))

analysis.release()

print("Post-Release\n")
print("DP mean of age: {0}".format(age_mean.value))
print("Privacy usage: {0}\n\n".format(analysis.privacy_usage))


Pre-Release

DP mean of age: None
Privacy usage: approximate {
  epsilon: 0.65
}



Post-Release

DP mean of age: 44.723527513068234
Privacy usage: approximate {
  epsilon: 0.65
}





As a result, a user will not know whether or not the validator will allow a proposed analysis until running `analysis.release()`.

In [11]:
''' incomplete analysis plan, but no release => no failure '''
with wn.Analysis() as analysis:
    # load data
    data = wn.Dataset(path = data_path, column_names = var_names)

    age_mean_fail = wn.dp_mean(data = wn.to_float(data['age']),
                          privacy_usage = {'epsilon': .65})

In [12]:
''' fails upon release '''
analysis.release()

Error: at node_id 5
Caused by: node specification LaplaceMechanism(LaplaceMechanism { privacy_usage: [PrivacyUsage { distance: Some(Approximate(DistanceApproximate { epsilon: 0.65, delta: 0.0 })) }] }):
Caused by: node specification Mean(Mean):
Caused by: data may contain nullity when non-nullity is required

